In [16]:
import pandas as pd

In [17]:
# Read the CSV file, skipping blank lines and bad lines
directory_path = r"C:\Users\LENOVO\1. Projects\BodyCote\Data\data_12122023.csv"
df = pd.read_csv(directory_path)

In [18]:
def data_transformation(df):
    directory_path = r"C:\Users\LENOVO\1. Projects\BodyCote\Data\data_12122023.csv"
    df = pd.read_csv(directory_path)
    # Convert TimeStamp column to datetime format
    df.loc[:, 'TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%d/%m/%Y %H:%M:%S')
    df.drop_duplicates(subset='TimeStamp', keep='last', inplace=True)
    # make time stamp as index and sort by the index
    df.set_index('TimeStamp', inplace=True)
    df.sort_index()
    # remove duplicates
    clean_data = df
    # return numeric dataset
    data_training = clean_data.select_dtypes(include=['float64', 'int64'])

    return clean_data, data_training

In [19]:
clean_data, data_training = data_transformation(df)

In [20]:
clean_data.head()

,Sensor_Name,fatigue_mmps,impact_mg0,friction_mg0,crest,Temperature_C,Temperature_F,status
TimeStamp,,,,,,,,
2023-11-01 02:18:58,V9 VVB001 Fan Motor,0.0,20.394324,10.197162,3.8,25.9,78.62,Device is OK
2023-11-01 02:48:38,V9 VVB001 Fan Motor,0.1,30.591486,10.197162,4.1,26.5,79.70,Device is OK
2023-11-01 02:49:38,V9 VVB001 Fan Motor,0.1,30.591486,10.197162,4.4,26.5,79.70,Device is OK
2023-11-01 05:36:46,V9 VVB001 Fan Motor,0.1,30.591486,10.197162,3.8,24.5,76.10,Device is OK
2023-11-01 05:37:47,V9 VVB001 Fan Motor,0.1,30.591486,10.197162,3.7,24.5,76.10,Device is OK


In [21]:
data_training.head()

,fatigue_mmps,impact_mg0,friction_mg0,crest,Temperature_C,Temperature_F
TimeStamp,,,,,,
2023-11-01 02:18:58,0.0,20.394324,10.197162,3.8,25.9,78.62
2023-11-01 02:48:38,0.1,30.591486,10.197162,4.1,26.5,79.70
2023-11-01 02:49:38,0.1,30.591486,10.197162,4.4,26.5,79.70
2023-11-01 05:36:46,0.1,30.591486,10.197162,3.8,24.5,76.10
2023-11-01 05:37:47,0.1,30.591486,10.197162,3.7,24.5,76.10


In [24]:
import joblib
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging
import os
from datasets import load_dataset, load_from_disk


In [ ]:
from dataclasses import dataclass
from pathlib import Path
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
import os
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

@dataclass(frozen=True)
class AnomalyDetectorConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    input_shape: tuple
    encoder_units_1_range: list
    encoder_units_2_range: list
    learning_rate_range: list
    batch_size_range: list
    epochs_range: list

class ConfigurationManager:
    # ... existing code ...

    def get_anomaly_detector_config(self) -> AnomalyDetectorConfig:
        config = self.config.anomaly_detector
        params = self.params.AnomalyDetectorArguments

        create_directories([config.root_dir])

        anomaly_detector_config = AnomalyDetectorConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            input_shape=params.input_shape,
            encoder_units_1_range=params.encoder_units_1_range,
            encoder_units_2_range=params.encoder_units_2_range,
            learning_rate_range=params.learning_rate_range,
            batch_size_range=params.batch_size_range,
            epochs_range=params.epochs_range
        )

        return anomaly_detector_config

class AnomalyDetectorTrainer:
    
    def __init__(self, config: AnomalyDetectorConfig):
        self.config = config
        self.model = None
    def preprocess_data(self):
        # Load your data here
        train, test = load_from_disk(self.config.data_path)

        # Normalize the data
        self.scaler = MinMaxScaler()
        X_train = self.scaler.fit_transform(train)
        X_test = self.scaler.transform(test)

        # Save the scaler
        scaler_filename = os.path.join(self.config.root_dir, "scaler_data.pkl")
        joblib.dump(self.scaler, scaler_filename)

        # Reshape inputs for LSTM [samples, timesteps, features]
        timesteps = 1  # Adjust based on your data
        features = X_train.shape[1]

        X_train = X_train.reshape((X_train.shape[0], timesteps, features))
        X_test = X_test.reshape((X_test.shape[0], timesteps, features))

        logging.info(f"Training data shape: {X_train.shape}")
        logging.info(f"Test data shape: {X_test.shape}")

        return X_train, X_test
    
    def create_model(self, input_shape, encoder_units_1, encoder_units_2, learning_rate):
        inputs = Input(shape=input_shape)
        x = LSTM(encoder_units_1, activation='relu', return_sequences=True)(inputs)
        x = LSTM(encoder_units_2, activation='relu', return_sequences=False)(x)
        x = RepeatVector(input_shape[0])(x)
        x = LSTM(encoder_units_2, activation='relu', return_sequences=True)(x)
        x = LSTM(encoder_units_1, activation='relu', return_sequences=True)(x)
        output = TimeDistributed(Dense(input_shape[1]))(x)
        model = Model(inputs=inputs, outputs=output)
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mae')
        return model

    def train(self):
        # Prepare data (load and preprocess data)
        # X_train, Y_train = load_your_data(self.config.data_path)

        # Define the grid search parameters
        param_grid = {
            'input_shape': [self.config.input_shape],
            'encoder_units_1': self.config.encoder_units_1_range,
            'encoder_units_2': self.config.encoder_units_2_range,
            'learning_rate': self.config.learning_rate_range,
            'batch_size': self.config.batch_size_range,
            'epochs': self.config.epochs_range
        }

        # Wrap Keras model for use with scikit-learn
        model = KerasRegressor(build_fn=self.create_model, verbose=0)

        # Create GridSearchCV
        grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
        grid_result = grid.fit(X_train, Y_train)

        # Summarize results and save the best model
        print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
        self.model = grid_result.best_estimator_.model
        self.model.save(os.path.join(self.config.root_dir, "anomaly_detector_model.h5"))

try:
    config = ConfigurationManager()
    anomaly_detector_config = config.get_anomaly_detector_config()
    anomaly_detector = AnomalyDetectorTrainer(config=anomaly_detector_config)
    anomaly_detector.train()
except Exception as e:
    raise e
